<a href="https://colab.research.google.com/github/Seongjin1225/AI_SCHOOL_9_DataTon/blob/main/Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


# Full Numbers

In [ ]:
import numpy as np
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('svhn', 'full_numbers', split='train')
test_dataset = load_dataset('svhn', 'full_numbers', split='test')

train_dataset[0]['image']
train_dataset[0]['digits']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/33402 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13068 [00:00<?, ? examples/s]

Generating extra split:   0%|          | 0/202353 [00:00<?, ? examples/s]

{'bbox': [[38, 1, 21, 40], [57, 3, 16, 40]], 'label': [4, 6]}

In [ ]:
# 이미지 자르기
image_list = []
label_list = []

for x in train_dataset:
    for i in range(len(x['digits']['bbox'])):
        x_min, y_min, width, height = x['digits']['bbox'][i]  # bounding box의 좌표를 가져옵니다.
        right = x_min + width
        lower = y_min + height
        image_list.append(x['image'].crop((x_min, y_min, right, lower)))
        label_list.append(x['digits']['label'][i])

# 이미지 저장 경로
save_train_dir = '/content/save/train-images'

for i, img in enumerate(image_list):
    # 라벨에 해당하는 디렉토리 경로
    label_dir = os.path.join(save_train_dir, str(label_list[i]))

    # 라벨 디렉토리가 없으면 생성
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    # 이미지 파일 경로
    img_path = os.path.join(label_dir, f'image_{i}.jpg')

    # 이미지 파일 저장
    img.save(img_path)


test_img_lst = []
test_label_lst = []

for data in test_dataset:
    for i in range(len(data['digits']['bbox'])):
        x, y, width, height = data['digits']['bbox'][i]  # bounding box의 좌표를 가져옵니다.
        right = x + width
        lower = y + height
        test_img_lst.append(data['image'].crop((x, y, right, lower)))
        test_label_lst.append(data['digits']['label'][i])

In [ ]:
# 테스트 이미지 자르기
image_test_list = []
label_test_list = []

for x in test_dataset:
    for i in range(len(x['digits']['bbox'])):
        x_min, y_min, width, height = x['digits']['bbox'][i]  # bounding box의 좌표를 가져옵니다.
        right = x_min + width
        lower = y_min + height
        image_test_list.append(x['image'].crop((x_min, y_min, right, lower)))
        label_test_list.append(x['digits']['label'][i])


# 이미지 저장 경로
save_test_dir = '/content/save/test-images'

for i, img in enumerate(image_test_list):
    # 라벨에 해당하는 디렉토리 경로
    label_dir = os.path.join(save_test_dir, str(label_test_list[i]))

    # 라벨 디렉토리가 없으면 생성
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    # 이미지 파일 경로
    img_path = os.path.join(label_dir, f'image_{i}.jpg')

    # 이미지 파일 저장
    img.save(img_path)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(directory=save_train_dir,
                                                    classes=[str(i) for i in range(10)],
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

validation_generator = test_datagen.flow_from_directory(directory=save_test_dir,
                                                        classes=[str(i) for i in range(10)],
                                                        target_size=(224,224),
                                                        batch_size=32,
                                                        subset='validation',
                                                        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(directory=save_test_dir,
                                                  classes=[str(i) for i in range(10)],
                                                  target_size=(224,224),
                                                  batch_size=64,
                                                  subset='training',
                                                  class_mode='categorical')


Found 73257 images belonging to 10 classes.
Found 5201 images belonging to 10 classes.
Found 20831 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.callbacks import Callback
from time import time
from tqdm import tqdm


class TqdmProgressCallback(Callback):

    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']
        self.start_time = time()
        self.times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time()
        print('\nEpoch %d/%d' % (epoch + 1, self.epochs))
        self.tqdm_bar = tqdm(total=self.params['steps'], position=0, leave=True)
        self.logs = {}

    def on_batch_end(self, batch, logs={}):
        self.tqdm_bar.update()
        self.logs = logs

    def on_epoch_end(self, epoch, logs={}):
        self.tqdm_bar.close()
        epoch_time = time() - self.epoch_start_time
        self.times.append(epoch_time)
        avg_time_per_epoch = sum(self.times) / len(self.times)
        remaining_time = avg_time_per_epoch * (self.epochs - epoch - 1)
        total_time = time() - self.start_time
        print(' - loss: %.4f - acc: %.4f - val_loss: %.4f - val_acc: %.4f - epoch_time: %.4f s - total_time: %.4f s - remaining_time: %.4f s' %
              (logs['loss'], logs['acc'], logs['val_loss'], logs['val_acc'], epoch_time, total_time, remaining_time))

In [ ]:
from keras.applications import DenseNet201

model_base = DenseNet201(weights='imagenet',
                      include_top=False,
                      input_shape=(224,224,3))

num = 1
for layer in model_base.layers:
    if num >= 3:
        layer.trainable=True
    else:
        layer.trainable=False
    num += 1

model_base.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_4 (ZeroPadd  (None, 230, 230, 3)          0         ['input_3[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_4[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

In [ ]:
model = Sequential()

model.add(model_base)

# classifier를 구현

model.add(Flatten())

# Hidden Layer
model.add(Dense(units=512,
                activation='relu'))

model.add(Dropout(rate=0.5))

# Output Layer
model.add(Dense(units=10,
                activation='softmax'))

# model.summary()

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 flatten (Flatten)           (None, 94080)             0         
                                                                 
 dense (Dense)               (None, 512)               48169472  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 66496586 (253.66 MB)
Trainable params: 66267530 (252.79 MB)
Non-trainable params: 229056 (894.75 KB)
_________________________________________________________________


In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=200,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=163,
                    verbose=0,
                    callbacks=[TqdmProgressCallback()]
                    )


Epoch 1/10


100%|██████████| 200/200 [04:21<00:00,  1.31s/it]


 - loss: 2.2913 - acc: 0.2686 - val_loss: 1.3826 - val_acc: 0.5497 - epoch_time: 261.6612 s - total_time: 261.7087 s - remaining_time: 2354.9505 s

Epoch 2/10


100%|██████████| 200/200 [02:15<00:00,  1.48it/s]


 - loss: 1.3816 - acc: 0.5383 - val_loss: 0.7807 - val_acc: 0.7568 - epoch_time: 135.3484 s - total_time: 397.0743 s - remaining_time: 1588.0383 s

Epoch 3/10


100%|██████████| 200/200 [02:15<00:00,  1.48it/s]


 - loss: 0.9626 - acc: 0.7036 - val_loss: 0.6514 - val_acc: 0.8020 - epoch_time: 135.2703 s - total_time: 532.3611 s - remaining_time: 1241.9863 s

Epoch 4/10


100%|██████████| 200/200 [02:15<00:00,  1.48it/s]


 - loss: 0.8177 - acc: 0.7552 - val_loss: 0.6697 - val_acc: 0.8008 - epoch_time: 135.4437 s - total_time: 667.8227 s - remaining_time: 1001.5854 s

Epoch 5/10


 77%|███████▋  | 154/200 [01:26<00:25,  1.80it/s]

In [ ]:
from tensorflow.keras.models import load_model

# model.save('/content/drive/MyDrive/DataTon/model/try_2(res50).h5')
model.save('/content/drive/MyDrive/DataTon/model/try.keras')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()

train_loss = history.history['loss']
valid_loss = history.history['val_loss']

train_acc = history.history['acc']
valid_acc = history.history['val_acc']

fig.add_subplot(2,1,1)
plt.plot(train_loss, label='train loss', color='r')
plt.plot(valid_loss, label='valid loss', color='b')
plt.legend()
plt.show()

fig.add_subplot(2,1,2)
plt.plot(train_acc, label='train acc', color='r')
plt.plot(valid_acc, label='valid acc', color='b')
plt.legend()
plt.show()


In [ ]:
min_train_loss = min(history.history['loss'])
min_valid_loss = min(history.history['val_loss'])

# 훈련 및 검증 정확도의 최대값
max_train_acc = max(history.history['acc'])
max_valid_acc = max(history.history['val_acc'])

# ResNet50의 최대 정확도 값
densenet201_max_acc = max(history.history['val_acc'])

print("Train Loss 최소값:", min_train_loss)
print("Valid Loss 최소값:", min_valid_loss)
print("Train Accuracy 최대값:", max_train_acc)
print("Valid Accuracy 최대값:", max_valid_acc)
print("DenseNet201 + classifier Max ACC :", densenet201_max_acc)

In [ ]:
# 파일명 : 숫자이미지가 들어 있는 파일명
# height: 이미지안에 들어 있는 숫자영역 사각형의 높이
# left: 이미지안에 들어 있는 숫자영역 사각형의 왼쪽 좌표
# top: 이미지안에 들어 있는 숫자영역 사각형의 위쪽 좌표
# width: 이미지안에 들어 있는 숫자영역 사각형의 너비
# label: 이미지안에 들어 있는 숫자영역 사각형안의 숫자에 대한 label
# 파일명	height	left	top	width	label

In [ ]:
# 테스트 파일 불러오기
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/DataTon/data.csv')
df.head(5)

In [ ]:
import os
from PIL import Image
# 이동할 폴더 경로 설정
folder_path = "/content/drive/MyDrive/DataTon/데이터톤 테스트셋"

# 폴더로 이동
directory = os.chdir(folder_path)

# 현재 디렉토리 안의 모든 파일과 디렉토리 리스트 가져오기
file_list = os.listdir(directory)
print(len(file_list))
print(sorted(file_list))

In [ ]:
import os
from PIL import Image
# 이동할 폴더 경로 설정
folder_path = "/content/drive/MyDrive/DataTon/데이터톤 테스트셋"

image_eval_list = []
label_eval_list = []

# PNG 파일만 필터링하여 출력하기
cnt=0
for k in range(1,240):
    filename = f'{k}.png'
    image_path = os.path.join(folder_path, filename)
    try:
        img = Image.open(image_path)

        # 이미지 데이터 처리

        num = len(df[df['파일명']==filename])
        for i in range(num):
            x_min, y_min, width, height = df['left'][cnt], df['top'][cnt], df['width'][cnt], df['height'][cnt]
            # print(x_min, y_min, width, height)
            right = x_min + width
            lower = y_min + height

            image_eval_list.append(img.crop((x_min, y_min, right, lower)))
            # print(df['label '][cnt])
            label_eval_list.append(df['label '][cnt])
            cnt+=1

        # 이미지 닫기
        img.close()

    except FileNotFoundError:
        cnt+=1

In [ ]:
# 이미지 저장 경로
save_eval_dir = '/content/save/eval-images'

for i, img in enumerate(image_eval_list):

    # 라벨에 해당하는 디렉토리 경로
    label_eval_dir = os.path.join(save_eval_dir, str(label_eval_list[i]))

    # 라벨 디렉토리가 없으면 생성
    if not os.path.exists(label_eval_dir):
        os.makedirs(label_eval_dir)

    # 이미지 파일 경로
    img_path = os.path.join(label_eval_dir, f'image_{i}.jpg')

    # 이미지 파일 저장
    img.save(img_path)

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# # 이미지 리스트 리사이즈
# resized_images = [img.resize((100, 200)) for img in image_list]

# # 이미지 데이터를 Numpy 배열로 변환
# image_array = np.array([image.img_to_array(img) for img in resized_images])

# # 레이블 데이터를 Numpy 배열로 변환
# label_array = np.array(label_list)

save_eval_dir = '/content/save/eval-images'
batch_size = 32

# ImageDataGenerator를 사용하여 데이터 증강 설정
eval_libdatagen = ImageDataGenerator(rescale=1/255)

eval_generator = eval_libdatagen.flow_from_directory(directory=save_eval_dir,
                                                      classes=[str(i) for i in range(10)],
                                                      target_size=(224, 224),
                                                      batch_size=batch_size,
                                                      # subset='validation',
                                                      class_mode='categorical'
                                                      )


In [ ]:
from tensorflow.keras.models import load_model
model_path = '/content/drive/MyDrive/DataTon/model/try.keras'
loaded_model = load_model(model_path)
loss, accuracy = loaded_model.evaluate(eval_generator)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')